In [1]:
!pip3 install opencv-python
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np

2022-11-21 19:38:58.487756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 19:38:59.401445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gennalll/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-21 19:38:59.401478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-21 19:38:59.515491: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already

In [2]:
im=cv2.imread('image/0a0ebb5f-f122-4d80-8fee-862bda74c369.png')
im2=cv2.imread('mask/0a0ebb5f-f122-4d80-8fee-862bda74c369.png')

[ WARN:0@4.830] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('image/0a0ebb5f-f122-4d80-8fee-862bda74c369.png'): can't open/read file: check file path/integrity
[ WARN:0@4.830] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('mask/0a0ebb5f-f122-4d80-8fee-862bda74c369.png'): can't open/read file: check file path/integrity


In [12]:
import os

input_dir = "image/"
target_dir = "mask/"
img_size = (512, 512)
num_classes = 3
batch_size = 32

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)


Number of samples: 3500
image/002dcdb5-34be-41bb-87f4-17c43ecb79a2.png | mask/002dcdb5-34be-41bb-87f4-17c43ecb79a2.png
image/005cb8b0-2e8b-4877-800f-786c6e8ad7fa.png | mask/005cb8b0-2e8b-4877-800f-786c6e8ad7fa.png
image/005df9a1-36c2-42d0-a402-8d72b0a33109.png | mask/005df9a1-36c2-42d0-a402-8d72b0a33109.png
image/006ebf59-9794-4595-85c4-beead171e1f5.png | mask/006ebf59-9794-4595-85c4-beead171e1f5.png
image/006ff0f9-6eb4-408a-bd63-7efedccec5ba.png | mask/006ff0f9-6eb4-408a-bd63-7efedccec5ba.png
image/00d16cc8-c5be-4c4a-97bc-cc3f38bc8ef8.png | mask/00d16cc8-c5be-4c4a-97bc-cc3f38bc8ef8.png
image/00dd4204-4ac6-4018-a888-e090114fe8fa.png | mask/00dd4204-4ac6-4018-a888-e090114fe8fa.png
image/00e53e12-c945-45f4-9d57-c4ec5327ba01.png | mask/00e53e12-c945-45f4-9d57-c4ec5327ba01.png
image/00f459b4-e020-4c34-9a31-cdeb9fdb2a20.png | mask/00f459b4-e020-4c34-9a31-cdeb9fdb2a20.png
image/010b10c7-4c20-4639-8327-1396371a4ac1.png | mask/010b10c7-4c20-4639-8327-1396371a4ac1.png


In [19]:
import random

# Split our img paths into a training and a validation set
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = train_input_img_paths, train_target_img_paths

val_gen =  val_input_img_paths, val_target_img_paths


In [20]:
from tensorflow.keras import layers


def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(image_size, 2)
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

 max_pooling2d_2 (MaxPooling2D)  (None, 32, 32, 256)  0          ['batch_normalization_6[0][0]']  
                                                                                                  
 conv2d_3 (Conv2D)              (None, 32, 32, 256)  33024       ['add_1[0][0]']                  
                                                                                                  
 add_2 (Add)                    (None, 32, 32, 256)  0           ['max_pooling2d_2[0][0]',        
                                                                  'conv2d_3[0][0]']               
                                                                                                  
 activation_7 (Activation)      (None, 32, 32, 256)  0           ['add_2[0][0]']                  
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 32, 32, 256)  590080     ['activation_7[0][0]']           
 ose)     

                                                                                                  
 conv2d_6 (Conv2D)              (None, 256, 256, 64  8256        ['up_sampling2d_5[0][0]']        
                                )                                                                 
                                                                                                  
 add_5 (Add)                    (None, 256, 256, 64  0           ['up_sampling2d_4[0][0]',        
                                )                                 'conv2d_6[0][0]']               
                                                                                                  
 activation_13 (Activation)     (None, 256, 256, 64  0           ['add_5[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_tr

In [21]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

In [23]:
model.fit(train_gen, epochs=1)

ValueError: in user code:

    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/gennalll/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        ra